## Sunday Notebook
### trying to measure min and max distance from home plate to the outfield fence

In [ ]:
## import libraries and read the kml file
### Taken from old geometry_workbook


from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
from area import area

file_path = ('data\kml\My Places_dec_9_22.kml')

with open(file_path) as file:

    xml_data = file.read()

# Initializing soup variables
soup = BeautifulSoup(xml_data, 'xml')

folders = soup.Document.Folder
list = soup.Document.Folder.find_all('Folder')
layers = soup.Document.Folder.Folder
polygons = soup.Document.Folder.Placemark.Polygon

## Create a dataframe to hold the data parsed from xml

df = pd.DataFrame(columns=['field', 'foul', 'fop'])



## Loop through the folders and extract the data


    
i = 0   

for i in range(len(list)):

    folders = list[i]
    field_name = folders.find('name').text
    foul = folders.find_all('coordinates')[0].text
    fop = folders.find_all('coordinates')[1].text

    row = {
        'field': field_name,
        'foul': foul,
        'fop': fop
    }

    i+=1

    df = df.append(row, ignore_index=True)

## Clean up the data

# remove new line and and space characters from coordinates
df = df.replace(r'\n','', regex=True) 
df = df.replace(r'\t','', regex=True) 


## Drop any duplicate rows
df = df.drop_duplicates(subset=['field'], keep='first')

## Drop and rows with empty fields
df = df[(df != 0).all(1)]


## remove numberic characters and . from field names
df['field'] = df['field'].str.replace(r'\d+', '')
df['field'] = df['field'].str.replace(r'\.', '')

In [ ]:


## Create a new column for home plate location by taking the first coordinate of the column

df['home_plate'] = df['fop'].str.split(' ').str[0]

## Create a new column for outfield_fence by dropping the first and last values in the fop column

df['outfield_fence'] = df['fop'].str.split(' ').str[1:-1]

df.head()


In [7]:
### 
#### Sunday 12/13/20
### going to try a new approach to measue the longest and shortest distance from home plate to the fop_line
### using a buffer around the home plate point and then finding the intersection of the buffer and the fop_line

### using havesine formula to calculate the distance between two points per this link
### https://towardsdatascience.com/calculating-distance-between-two-geolocations-in-python-26ad3afe287b

# import haversine as hs

# ## get the location of the foul poles by getting the first and last set of coordinates from the outfield_fence column
# ## put the coordinates into new columns fp1 and fp2
# df['fp1'] = df['outfield_fence'].str[0]
# df['fp2'] = df['outfield_fence'].str[-1]

# # convert the coordinates to a tuple
# df['fp1'] = df['fp1'].str.split(',').apply(lambda x: (float(x[0]), float(x[1])))
# df['fp2'] = df['fp2'].str.split(',').apply(lambda x: (float(x[0]), float(x[1])))
# df['home_plate'] = df['home_plate'].str.split(',').apply(lambda x: (float(x[0]), float(x[1])))


# df.head()


# convert the outfield_fence column to pairs of coordinates
# then calculate the distance between home_plate coordinates and each vertex
# then find the max and min distance


# create a new column to hold the distance from home plate to each vertex
# df['distance'] = 0



# df.head()
x = 2


loc1 = df['home_plate'][x]
loc2 = (df['fp1'][x])

hs.haversine(loc1, loc2, unit='ft')

KeyError: 'home_plate'

In [11]:
### Trying a new plotting method.

# test run a group of 10 fields to try to caculate distance from home plate to the fop_line

## import libraries and read the kml file
### Taken from old geometry_workbook


from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
from area import area

file_path = ('data\kml\BB_colleges_12_17.kml')

with open(file_path) as file:

    xml_data = file.read()

# Initializing soup variables
soup = BeautifulSoup(xml_data, 'xml')

folders = soup.Document.Folder
list = soup.Document.find_all('Folder')
layers = soup.Document.Folder.Folder
polygons = soup.Document.Folder.Placemark.Polygon

## Create a dataframe to hold the data parsed from xml

df = pd.DataFrame(columns=['field', 'foul', 'outfield_fence'])



## Loop through the folders and extract the data


    
i = 0   

for i in range(len(list)):

    # folders = list[i]
    field_name = folders.name
    foul = list.find_all('coordinates')[0]
    fence = list.find_all('coordinates')[1]

    row = {
        'field': field_name,
        'foul': foul,
        'outfield_fence': fence
    }

    i+=1

    df = df.append(row, ignore_index=True)



AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [5]:
### take the first pair of coordinates from the foul column and use it as the home plate location
df['foul'] = df['foul'].to_string()

# remove new line and and space characters from coordinates
df = df.replace(r'\n','', regex=True) 
df = df.replace(r'\t','', regex=True) 


df['home_plate'] = df['foul'].str.split(' ').str[1]

df.head()


,field,foul,outfield_fence,home_plate
0,Northwood University - college,"0 [\n -84.2801942,43.6331651...","[\n -84.2801701,43.634062,0\n ...",
1,Northwood University - college,"0 [\n -84.2801942,43.6331651...","[\n -84.2801701,43.634062,0\n ...",
2,Northwood University - college,"0 [\n -84.2801942,43.6331651...","[\n -84.2801701,43.634062,0\n ...",
3,Northwood University - college,"0 [\n -84.2801942,43.6331651...","[\n -84.2801701,43.634062,0\n ...",
4,Northwood University - college,"0 [\n -84.2801942,43.6331651...","[\n -84.2801701,43.634062,0\n ...",


In [ ]:

import haversine as hs

### measure the distance from home_plate to each vertice in the outfield_fence column

for x in range(len(df)):

    for y in range(len(df['outfield_fence'][x])):

        loc1 = df['home_plate'][x]
        loc2 = df['outfield_fence'][x][y]

        df['distance'][x] = hs.haversine(loc1, loc2, unit='ft')

        print(df['distance'][x])

df.head()


# home_plate = df['home_plate'][x]


# hs.haversine(loc1, loc2, unit='ft')


